In [1]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
torch.tensor([0.75,0.5,0.25],device='cpu')

tensor([0.7500, 0.5000, 0.2500])

In [3]:
img=plt.imread(r'F:\study\ml\LM\image\13\catdog.jpg')
h,w=img.shape[:2]
print(h,w)

561 728


In [4]:
def multibox_prior(data,sizes,ratios):
    in_height,in_width=data.shape[-2:]
    num_sizes,num_ratios=len(sizes),len(ratios)
    boxes_per_pixel=(num_sizes+num_ratios-1)
    size_tensor=torch.tensor(sizes)
    ratio_tensor=torch.tensor(ratios)
    
    offset_h,offset_w=0.5,0.5
    step_h=1.0/in_height
    step_w=1.0/in_width
    
    center_h=(torch.arange(in_height)+offset_h)*step_h
    center_w=(torch.arange(in_width)+offset_w)*step_w
    shift_y,shift_x=torch.meshgrid(center_h,center_w)
    shift_y,shift_x=shift_y.reshape(-1),shift_x.reshape(-1)
    
    w=torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]),size_tensor[0]*torch.sqrt(ratio_tensor[1:]))) * in_height/in_width
    h=torch.cat((size_tensor / torch.sqrt(ratio_tensor[0]),size_tensor[0] / torch.sqrt(ratio_tensor[1:])))
    
    anchor_manipulations=torch.stack((-w,-h,w,h)).T.repeat(in_height*in_width ,1 )/2
    out_grid=torch.stack([shift_x,shift_y,shift_x,shift_y],dim=1).repeat_interleave(boxes_per_pixel,dim=0)
    output=out_grid+anchor_manipulations
    return output.unsqueeze(0)    

In [5]:
X = torch.rand(size=(1, 3, h, w))
Y = multibox_prior(X, sizes=[0.75, 0.5, 0.25], ratios=[1, 2, 0.5])
Y.shape

D:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 2042040, 4])

In [6]:
def bbox_to_rect(bbox,color):
    return plt.Rectangle(xy=(bbox[0],bbox[1]),width=bbox[2]-bbox[0],height=bbox[3]-bbox[1],fill=False,edgecolor=color,linewidth=2)

In [7]:
def show_bboxes(axes,bboxes,labels=None,colors=None):
    def _make_list(obj,default_values=None):
        if obj is None:
            obj=default_values
        elif not isinstance(obj,(list,tuple)):
            obj=[obj]
        return obj
    
    labels=_make_list(labels)
    colors=_make_list(colors,['b','g','r','m','c'])
    plt.imshow(axes)
    for i ,bbox in enumerate(bboxes):
        color=colors[i % len(colors)]
        rect=bbox_to_rect(bbox.detach().numpy(),color)
        plt.gca().add_patch(rect)
        if labels and len(labels) > i:
            text_color='k' if color=='w' else 'w'
            plt.text(rect.xy[0],rect.xy[1],labels[i],va='center',ha='center',fontsize=9,color=text_color,bbox=dict(facecolor=color,lw=0))
    

In [8]:
def display_anchors(fmap_w,fmap_h,s):
    plt.figure(figsize=(8,6))
    fmap=torch.zeros((1,10,fmap_h,fmap_w))
    anchors=multibox_prior(fmap,sizes=s,ratios=[1,2,0.5])
    print('anchors shape : ',anchors.shape)
    bbox_scale=torch.tensor((w,h,w,h))
    print('bbox_scale shape: ',bbox_scale.shape)
    print('bbox_scale : ',bbox_scale)
    show_bboxes(img,anchors[0]*bbox_scale)
    

In [9]:
torch.tensor([0.75,0.5,0.25],device='cpu')

tensor([0.7500, 0.5000, 0.2500])

In [10]:
torch.tensor([0.75,0.5,0.25],device='cpu')

tensor([0.7500, 0.5000, 0.2500])

In [11]:
display_anchors(fmap_w=4,fmap_h=4,s=[0.15])

anchors shape :  torch.Size([1, 48, 4])
bbox_scale shape:  torch.Size([4])
bbox_scale :  tensor([728, 561, 728, 561])


In [13]:
display_anchors(fmap_w=2,fmap_h=2,s=[0.3])

anchors shape :  torch.Size([1, 12, 4])
bbox_scale shape:  torch.Size([4])
bbox_scale :  tensor([728, 561, 728, 561])


In [14]:
display_anchors(fmap_w=1,fmap_h=1,s=[0.8])

anchors shape :  torch.Size([1, 3, 4])
bbox_scale shape:  torch.Size([4])
bbox_scale :  tensor([728, 561, 728, 561])
